# Exemple d'algorithmes génétiques avec un master mind 
## Objectif :
présenter simplement les algorithmes génétiques à l'aide d'un exemple simple, un master mind !
Ce fichier sera le premier d'une songue série de notebooks sur les algorithmes génétiques. 
Il pourra m'aider à l'analyse des différentes techniques et à leurs cas d'utilisation.
L'objectif est pour la machine de deviner une combinaison en un minimum d'iterations. 
Moins il y a d'iterations, plus l'algorithme est performant.Dans le mastermind original, l'utilisateur a perdu s'il ne trouve pas la combinaison gagnante au bout d'un certain nombre d'iterations. 
Dans l'exemple ici, la machine ne peut pas perdre. L'objectif était de tester la robustesse des algorithmes.
## Règle du mastermind
Le mastermind se joue à 2 joueurs, sur un plateau de 4 colonnes et de 12 rangés et se joue avec des billes de 4 couleurs différentes.Le mastermind possède une combinaison secrète de 4 billes de couleurs, et l'objectif du joueur est de deviner avant la 13e tentative la combinaison secrète. 
Pour l'aider, il lui est révélé né nombre de billes posées au bon endroit et de la bonne couleur, sans lui reveler lesquels sont correctes.
## Ce qui est présenté dans ce notebook
- La notion de fitness
- La notion de population
- La notion d'individus
- La notion de chromosomes
- La notion de genes 
## Concepts avancée dans ce notebook
- Les types de sélections    
    - Sélection par roulette    
    - Sélection par rang    
    - Sélection par tournoi    
    - Sélection aléatoire
- La notion d'élitisme
- Les types de reproduction d'individus    
    - One point cross-over    
    - Two-point cross-over    
    - Uniforme Cross-Over

In [1]:
# Importation des librairies utilisés
import random
import numpy as np

defaultProbaPerLetter = 10
batch_size = 30

In [2]:
# Definition du plateau
class Plateau:
    def __init__(self, secret):
        self.secret = secret
        self.secretLen = len(secret);
        #initialisation des genes (probabilité de 10 chaqu'un)
        self.caracteres = "azertyuiopqsdfghjklmwxcvbn0123456789_'";
        self.genesDict = []
        self.totalGenesCount = defaultProbaPerLetter*len(self.caracteres)
        # remplissage du tableau de prédiction avec la valeur par defaut.
        for i in range (0,self.secretLen):
            self.genesDict.append({});
            for l in self.caracteres:
                self.genesDict[i][l] = defaultProbaPerLetter;
        
    def printDict(self):
        print(self.genesDict)
        
    '''
    Cette fonction sert à mémoriser les résultats générés précedement. il enregistre pour chaque 
    case les résultats obtenus lors des precedentes tentatives. Cela permet à l'algorithme d'affiner lentement 
    sa prédiction pour tendre vers la bonne solution.
    '''
    def evolveGenes(self, individu, fitness):
        if (fitness == 0):
            return;
        index = 0
        for l in individu:
            self.genesDict[index][l]+= fitness
            index+=1;
        self.totalGenesCount += fitness

Il existe aussi une methode appelée uniform crossover, qui change aléatoirement les genes, dans "paquets ou coupures"distinctes. Un exemple avec les deux parents ----- et +++++ peut donner -+-+-. 
__Cet exemple n'est pas encore implémenté dans l'algorithme.__
<hr>
Voici la definition du joueur.

In [3]:
# Definition du joueur. 
# Il va jouer un coup, et le changer (muter) pour chercher une meilleur combinaision
class Joueur:
    def __init__(self, plateau):
        self.p = plateau;
        self.crossoverOne = True;
        
    # Generation d'un individu aléatoire.
    def generateRandomPlay(self):
        individu = "";
        for i in range(0,self.p.secretLen):
            lettre = p.caracteres[random.randint(0, len(p.caracteres)-1)];
            individu += lettre
        return individu;
    
    # generation d'un individu dont les genes sont probabilisé par les résultats précedents
    def generatePlay(self):
        individu = "";
        for i in range(0,self.p.secretLen):
            letterIndex = random.randint(0, p.totalGenesCount)
            for l in p.genesDict[i]:
                if (letterIndex - int(p.genesDict[i][l]) <= 0):
                    individu+=l
                    break
                else:
                    letterIndex -= p.genesDict[i][l]
        return individu;
    
    # generation d'une population de 10 individus (par defaut) avec des genes aléatoires.
    def generatePopulation(self):
        pop = [];
        for i in range(0,batch_size):
            play = self.generateRandomPlay();
            pop.append(play);
        return pop;
    
    '''
    Cette fonction genere des enfants qui possede une partie des genes de son parent.
    la generation des genes des enfant est généré par un one point crossover, ou un two point crossover.
    Le principe de mutation n'est pas introduit ici. il a pour principe de changer au hasard un gene sans 
    tenir compte de sa probabilité. Les mutations doivent etre rarent, et permettent de limiter la convergence
    de l'algorithme.
    '''
    def generatePopulationFromParent(self, parent):
        pop = [];
        for i in range(0,batch_size):
            play = self.generatePlay();
            if (self.crossoverOne):
                play = self.onePointCrossover(play);
            else:
                play = self.twoPointCrossover(play);
            pop.append(play);
        return pop;
        
    '''
    La fitness est la valeur utilisée par l'algorithme pour estimer ou noter le resultat d'un individu
    sur l'objectif à atteindre. Dans notre cas, chaque lettre correctement placée compte comme 1 point.
    La fitness maximale est donc la taille du mot secret !
    '''
    def getFitness(self, individu):
        fitness = 0;
        for i in range(0,self.p.secretLen):
            if(self.p.secret[i] == individu[i]):
                fitness += 1;
        return fitness;
    
    # retourne l'index dans la population de l'element avec la meilleur fitness 
    # (la plus haute correspondence au mot secret)
    def getFittestIndex(self, population):
        index = 0
        fittest = self.getFitness(population[index])
        for i in range(0,len(population)):
            fit = self.getFitness(population[i])
            if (fittest < fit):
                index = i;
                fittest = fit;
        return index;
    
    ''' 
    Le one point crossover est une methode de generation d'enfant.
    Admettons un parent avec le chromosome (ensemble de genes) suivant : '-----''
    Et admettons un individu généré avec les genes suivant : '+++++'
    Le one point crossover va generer l'enfant des deux parents en prennant 
    un index aléatoirement, puis en melangeant les genes du premier parent avec le second
    sur cet index.
    exemple avec l'index 2, l'enfant des deux parents ci dessus donne : '--+++'.
    '''
    def onePointCrossover(self, individu):
        side = random.randint(0, 1)
        i = random.randint(1,self.p.secretLen-1)
        play = self.generateRandomPlay()
        if (side == 0):
            ret = individu[:i] + play[:self.p.secretLen-i]
        else:
            ret = play[:i] + individu[i-self.p.secretLen:] 
        return ret;
        
    '''
    Le two point crossover sert egalement à melanger les genes de deux parents, mais cette fois-çi avec deux
    index.
    Donc pour les mêmes parents ----- et +++++ avec les index 2 et 4, l'enfant donnera : --++-.
    '''
    def twoPointCrossover(self, individu):
        i = random.randint(0,self.p.secretLen-1)
        size = random.randint(1,self.p.secretLen-i)
        play = self.generateRandomPlay()
        ret = individu[0:i] + play[i:i+size] + individu[i+size:self.p.secretLen]
        return ret;

In [4]:
# Initialisation du jeu
# Definir 5 combinaisons de 38 possibilités correspond à une chance sur 79.2 millions de tomber sur la bonne.
p = Plateau("wha0u");
j = Joueur(p);


Ici, nous allons selectionner le meilleur parent, et le faire se reproduire pour former la prochaine generation. C'est à mon sens la meilleure solution dans ce cas, car il n'existe qu'une seule solution, qui est accessible via un seul chemin. Résuire le nombre de parents permet donc de converger plus rapidement vers la meilleure solution.

Voici l'algorithme principal de la résolution du mastermind. un bref résumé de l'algorithme donne :
- Création d'une population aléatoire
- Calcul de la fitness de la population, et récuperation du meilleur individu
- Jusqu'a ce que la solution soit trouvé, faire :
    - Selectionner un parent depuis la population, 
    - Appliquer un crossover et generer une nouvelle population
    - Calculer la fitness de la nouvelle population

In [5]:
# algorithme principal :
#création d'une population de 10 individus pour le demarrage :
pop = j.generatePopulation();
fittestIndex = j.getFittestIndex(pop)
fitness = j.getFitness(pop[fittestIndex])
print(pop[fittestIndex] + "   --fitness : "+str(fitness));
p.evolveGenes(pop[fittestIndex], fitness)
i=1;
while (fitness < p.secretLen):
    pop = j.generatePopulationFromParent(pop[fittestIndex])
    fittestIndex = j.getFittestIndex(pop)
    fitness = j.getFitness(pop[fittestIndex])
    p.evolveGenes(pop[fittestIndex], fitness)
    print(pop[fittestIndex] + "   --fitness : "+str(fitness));
    i+=1;

print("!! Trouvé !! - "+ pop[fittestIndex]+"  //  tentatives : "+str(i))

tjs09   --fitness : 1
lgi0w   --fitness : 1
i7_0u   --fitness : 2
ghwg2   --fitness : 1
w5106   --fitness : 2
ihm0z   --fitness : 2
w4npv   --fitness : 1
ae50u   --fitness : 2
6h075   --fitness : 1
voay'   --fitness : 1
5hen2   --fitness : 1
xp80e   --fitness : 1
thnzu   --fitness : 2
f3p02   --fitness : 1
w45ti   --fitness : 1
wl20g   --fitness : 2
cja6e   --fitness : 1
0gagd   --fitness : 1
ooz8u   --fitness : 1
thlch   --fitness : 1
kha95   --fitness : 2
yhu09   --fitness : 2
38v0w   --fitness : 1
5'w9u   --fitness : 1
w'ap0   --fitness : 2
97a5x   --fitness : 1
mfl0y   --fitness : 1
ns00z   --fitness : 1
fhab6   --fitness : 2
wvalm   --fitness : 2
zhaho   --fitness : 2
8i50u   --fitness : 2
6yakc   --fitness : 1
w4kub   --fitness : 1
nam0'   --fitness : 1
0hn7c   --fitness : 1
qqack   --fitness : 1
2'aik   --fitness : 1
_sn9u   --fitness : 1
wh4p3   --fitness : 2
4y306   --fitness : 1
nh6w'   --fitness : 1
90z06   --fitness : 1
31m6u   --fitness : 1
8km0f   --fitness : 1
'hd0y   --

pma0u   --fitness : 3
whava   --fitness : 3
whaos   --fitness : 3
wha0o   --fitness : 4
whavu   --fitness : 4
whap8   --fitness : 3
k2a0u   --fitness : 3
zoa0u   --fitness : 3
wta0j   --fitness : 3
wham4   --fitness : 3
pha06   --fitness : 3
7ha0q   --fitness : 3
mna0u   --fitness : 3
whh0x   --fitness : 3
wsa01   --fitness : 3
wha0o   --fitness : 4
'ha0y   --fitness : 3
'6a0g   --fitness : 2
rha0f   --fitness : 3
1ha08   --fitness : 3
xha0n   --fitness : 3
wha9x   --fitness : 3
wha0u   --fitness : 5
!! Trouvé !! - wha0u  //  tentatives : 535


Dans l'algorithme principal au dessus, La selection du parent se fait simplement par le fait que le meilleur l'emporte. Cependant il existe plusieurs autres types de selection de parents. Nous allons en voir quelques exemples en dessous.
<hr>

# La selection par roulette
Le principe est simple. Les individus avec le plus de fitness ont plus de chance d'etre gardé pour la prochaine production de la population. illustration simple en image :

<img src="https://www.researchgate.net/profile/Manfred_Breit/publication/237507026/figure/fig7/AS:298797780488200@1448250350801/Roulette-wheel-selection-using-fitness-weighted-probability.png" alt="roulette wheel selection;">

[Source de l'image](https://www.researchgate.net/figure/Roulette-wheel-selection-using-fitness-weighted-probability_fig7_237507026)

Voici un algorithme pour le mettre en place :

In [6]:
pop = j.generatePopulation();
fittestIndex = j.getFittestIndex(pop)
fitness = j.getFitness(pop[fittestIndex])
print(pop[fittestIndex] + "   --fitness : "+str(fitness));
p.evolveGenes(pop[fittestIndex], fitness)
parentIndex = fittestIndex;
i=1;
while (fitness < p.secretLen):
    pop = j.generatePopulationFromParent(pop[parentIndex])
    fittestIndex = j.getFittestIndex(pop)
    fitness = j.getFitness(pop[fittestIndex])
    p.evolveGenes(pop[fittestIndex], fitness)
    print(pop[fittestIndex] + "   --fitness : "+str(fitness));
    i+=1;
    # calcul du total de fitness: 
    totalFitness = 0
    for J in range(0,len(pop)):
        totalFitness += j.getFitness(pop[J])
    #nombre aléatoire entier entre 0 et totalFirness :
    indexParent = random.randint(0,totalFitness);
    #Trouver l'index de l'individu à garder :
    parentIndex = 0
    for J in range(0,len(pop)):
        if (totalFitness <= 0):
            parentIndex = J;
            break;
        totalFitness -= j.getFitness(pop[J])

print("!! Trouvé !! - "+ pop[fittestIndex]+
      "  //  tentatives : "+str(i)+"  // nombre d'individus testé : "+str(i*batch_size))

b1'02   --fitness : 1
kha0g   --fitness : 3
whn0n   --fitness : 3
whay8   --fitness : 3
2hi0u   --fitness : 3
whaei   --fitness : 3
wha0o   --fitness : 4
wha54   --fitness : 3
2ha0b   --fitness : 3
xhan5   --fitness : 2
6haau   --fitness : 3
o5a0u   --fitness : 3
wha77   --fitness : 3
oha0h   --fitness : 3
ch20u   --fitness : 3
wha0n   --fitness : 4
wha0q   --fitness : 4
6ha0u   --fitness : 4
wha0y   --fitness : 4
qhd0u   --fitness : 3
73a0u   --fitness : 3
nha05   --fitness : 3
wha0r   --fitness : 4
dha0f   --fitness : 3
mha0z   --fitness : 3
79a0u   --fitness : 3
nhaeu   --fitness : 3
wha0d   --fitness : 4
8hagu   --fitness : 3
vh70u   --fitness : 3
pea0u   --fitness : 3
3ha0u   --fitness : 4
wha0r   --fitness : 4
xia0u   --fitness : 3
i3a0u   --fitness : 3
wza0x   --fitness : 3
wha0s   --fitness : 4
wha2h   --fitness : 3
3ha9u   --fitness : 3
b2a0u   --fitness : 3
wna08   --fitness : 3
wha04   --fitness : 4
8ha0q   --fitness : 3
wha11   --fitness : 3
0ha6j   --fitness : 2
wha2g   --

<hr>

# Sélection par rang
<img src="https://www.tutorialspoint.com/genetic_algorithms/images/rank_selection.jpg" alt="rank selection;">
Avec cette selection, on garde un nombre fixe d'individus issu de la population ayant la meilleur fitness. une fois ces individus en nôtre pocession, on en selectionne un au hasard, puis on regénere la population.

Cette methode fonctionne avec des elements posedant une fitness negative, mais à l'inconvenient de perdre en selection naturelle rapidement, et ainsi donc privilegier les mauvais parents lors de la génération.

exemple d'algorithme ci dessous :

In [7]:
#TODO
pop = j.generatePopulation()
fittestIndex = j.getFittestIndex(pop)
fitness = j.getFitness(pop[fittestIndex])
print(pop[fittestIndex] + "   --fitness : "+str(fitness))
p.evolveGenes(pop[fittestIndex], fitness)
parentIndex = fittestIndex
i=1;
while (fitness < p.secretLen):
    pop = j.generatePopulationFromParent(pop[parentIndex])
    fittestIndex = j.getFittestIndex(pop)
    fitness = j.getFitness(pop[fittestIndex])
    p.evolveGenes(pop[fittestIndex], fitness)
    print(pop[fittestIndex] + "   --fitness : "+str(fitness))
    i+=1;
    # calcul des n meilleurs individus de la population:
    n = 5
    arr = np.array([])
    for ind in range(0,len(pop)):
        arr = np.append(arr,j.getFitness(pop[ind]))
    #Récuperation des meilleurs index du tableau
    sortedArr = (-arr).argsort()[:n]
    # choix du prochain parent
    parentIndex = sortedArr[random.randint(0,n-1)]

print("!! Trouvé !! - "+ pop[fittestIndex]+
      "  //  tentatives : "+str(i)+"  // nombre d'individus testé : "+str(i*batch_size))

tg40'   --fitness : 1
wh507   --fitness : 3
6ha0c   --fitness : 3
whab8   --fitness : 3
esa0u   --fitness : 3
wh60'   --fitness : 3
pho0u   --fitness : 3
whagu   --fitness : 4
phw0u   --fitness : 3
wha8d   --fitness : 3
wha0u   --fitness : 5
!! Trouvé !! - wha0u  //  tentatives : 11  // nombre d'individus testé : 330


<hr>

# Sélection par tournoi
<img src="https://www.tutorialspoint.com/genetic_algorithms/images/tournament_selection.jpg" alt="tournement selection;">
Avec cette selection, On selectionne K nombre d'individus, puis on les fait "se battre" entre eux, de sorte à tirer les meilleurs.

exemple d'algorithme ci dessous :

In [8]:
pop = j.generatePopulation()
fittestIndex = j.getFittestIndex(pop)
fitness = j.getFitness(pop[fittestIndex])
print(pop[fittestIndex] + "   --fitness : "+str(fitness))
p.evolveGenes(pop[fittestIndex], fitness)
parentIndex = fittestIndex
i=1;
while (fitness < p.secretLen):
    pop = j.generatePopulationFromParent(pop[parentIndex])
    fittestIndex = j.getFittestIndex(pop)
    fitness = j.getFitness(pop[fittestIndex])
    p.evolveGenes(pop[fittestIndex], fitness)
    print(pop[fittestIndex] + "   --fitness : "+str(fitness))
    i+=1;
    # Selection des n maximum challengers de la population:
    n = 5
    arr = []
    indArr = []
    for ind in range (0,n):
        x = random.randint(0,batch_size-1)
        arr.append(j.getFitness(pop[x]))
        indArr.append(x)
    
    # Récuperation de l'index du parent
    winnerIndex = arr.index(max(arr))
    parentIndex = indArr[winnerIndex]
    
print("!! Trouvé !! - "+ pop[fittestIndex]+
      "  //  tentatives : "+str(i)+"  // nombre d'individus testé : "+str(i*batch_size))

44j04   --fitness : 1
ghacu   --fitness : 3
whv0u   --fitness : 4
6ha0u   --fitness : 4
w2z0u   --fitness : 3
_2a0u   --fitness : 3
hfa0u   --fitness : 3
wha2n   --fitness : 3
yha0z   --fitness : 3
wha0i   --fitness : 4
lha0u   --fitness : 4
wha02   --fitness : 4
wha13   --fitness : 3
fhg0u   --fitness : 3
wha01   --fitness : 4
'ha0u   --fitness : 4
_ua0u   --fitness : 3
wh7jr   --fitness : 2
wha0q   --fitness : 4
q1a0u   --fitness : 3
7ha0u   --fitness : 4
lha0m   --fitness : 3
wha0l   --fitness : 4
'ha0q   --fitness : 3
_ha0m   --fitness : 3
7ha00   --fitness : 3
wha0q   --fitness : 4
wha0w   --fitness : 4
whaj6   --fitness : 3
jha0u   --fitness : 4
wha05   --fitness : 4
4ha0u   --fitness : 4
lha0u   --fitness : 4
w0al6   --fitness : 2
whao0   --fitness : 3
4ha0o   --fitness : 3
whaxw   --fitness : 3
wha0r   --fitness : 4
yha0q   --fitness : 3
wh10s   --fitness : 3
whaap   --fitness : 3
whaoz   --fitness : 3
wha0l   --fitness : 4
lha03   --fitness : 3
jha0x   --fitness : 3
rha01   --